# Loading Python Modules

In [193]:
import numpy as np
import pandas as pd
import os
import re

# Specify git executable file for GitPython in Jupyter Notebook (In IDE, it can still work without this line.)
os.environ["GIT_PYTHON_GIT_EXECUTABLE"] = "C:\Program Files\Git\cmd\git.exe"

import git
from git import RemoteProgress

from git import Repo
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [194]:
# Colour

In [195]:
BLUE   = '\033[94m'
GREEN  = '\033[92m'
ORANGE = '\033[93m'
RED    = '\033[91m'
ENDC   = '\033[0m'

Cyan = '\033[96m'
White = '\033[97m'
Yellow = '\033[93m'
Magenta = '\033[95m'
Grey = '\033[90m'

# Create Object

In [352]:
vuln0_ID = "CVE"
vuln0_project_link = "https://github.com/spring-projects/spring-framework"
vuln0_local_link = "spring-framework"
vuln0_fixing_commit = "a95c3d820dbc4c3ae752f1b3ee22ee860b162402"
vuln0_repo = Repo(vuln0_local_link)
vuln0 = {"id": vuln0_ID, "project_link": vuln0_project_link, "local_link": vuln0_local_link, "fixing_commit": vuln0_fixing_commit, "repo": vuln0_repo}

# All Vulnerabilities as Dicts

In [353]:
vulns_info = [vuln0]

## Getting Affected Files

In [354]:
affected_files = {}
for vuln in vulns_info:
    diff_vuln = vuln["repo"].git.diff("--name-only", vuln["fixing_commit"], vuln["fixing_commit"]+"^1").splitlines()
    affected_files[vuln["id"]] = diff_vuln

In [355]:
for commit in list(git.objects.commit.Commit.iter_items( vuln["repo"], vuln["fixing_commit"])):
    if commit.hexsha == vuln["fixing_commit"]:
        affected_files1 = commit.stats.files;
        print("Changes:" + Yellow)
        print(affected_files1)

Changes:
{'spring-web/src/main/java/org/springframework/http/converter/json/MappingJackson2HttpMessageConverter.java': {'insertions': 1, 'deletions': 0, 'lines': 1}, 'spring-web/src/main/java/org/springframework/web/accept/ContentNegotiationManagerFactoryBean.java': {'insertions': 5, 'deletions': 1, 'lines': 6}, 'spring-web/src/main/java/org/springframework/web/accept/PathExtensionContentNegotiationStrategy.java': {'insertions': 2, 'deletions': 2, 'lines': 4}, 'spring-web/src/main/java/org/springframework/web/util/UrlPathHelper.java': {'insertions': 1, 'deletions': 1, 'lines': 2}, 'spring-web/src/main/java/org/springframework/web/util/WebUtils.java': {'insertions': 7, 'deletions': 4, 'lines': 11}, 'spring-web/src/test/java/org/springframework/http/converter/json/MappingJackson2HttpMessageConverterTests.java': {'insertions': 2, 'deletions': 2, 'lines': 4}, 'spring-web/src/test/java/org/springframework/web/accept/ContentNegotiationManagerFactoryBeanTests.java': {'insertions': 65, 'deleti

# Within Scope

In [246]:
import io
import sys

full_lines = []
add_lines  = []
sub_lines  = []
blame_lines= []

diff_data = vuln["repo"].git.diff(vuln["fixing_commit"] + "^", vuln["fixing_commit"]).splitlines()

commit = vuln["repo"].commit(vuln["fixing_commit"])

for line in diff_data:
    if line.startswith("++") or line.startswith("--"):
        continue
        
    if line.startswith("+"):
        add_lines.append(line)
    if line.startswith("-"):
        sub_lines.append(line)  


for affected_file in affected_files1:
    print("\n");
    print(RED + "--- File: " + RED + affected_file + " ---")
    searchScope = False
    
    targetfile = commit.tree / affected_file
    with io.BytesIO(targetfile.data_stream.read()) as f:
        full_lines = f.read().decode('utf-8').splitlines()

    print("addlines[0] = ", add_lines[0])

    for full_line in full_lines:
        if len(add_lines) == 0:
            break
            
        if len(full_line.lstrip()) == len(full_line):
            searchScope = False
       
        elif "}" in full_line:
            searchScope = False            
            
        if add_lines[0][1:] == full_line:
            searchScope = True
            print("=>" + BLUE, full_line, ENDC + "<=")
            add_lines.pop(0)  
        else:
            if searchScope == True:
                print("=>" + ORANGE + full_line + ENDC + "<=")
                blame_lines.append(full_line)
#             else:
#                 print("[" + full_line + "]")
       

print("should be empty: ", add_lines) 



--- File: spring-messaging/src/main/java/org/springframework/messaging/simp/broker/DefaultSubscriptionRegistry.java ---
addlines[0] =  + * Copyright 2002-2018 the original author or authors.
=>  * Copyright 2002-2018 the original author or authors. <=
=> *<=
=> * Licensed under the Apache License, Version 2.0 (the "License");<=
=> * you may not use this file except in compliance with the License.<=
=> * You may obtain a copy of the License at<=
=> *<=
=> *      http://www.apache.org/licenses/LICENSE-2.0<=
=> *<=
=> * Unless required by applicable law or agreed to in writing, software<=
=> * distributed under the License is distributed on an "AS IS" BASIS,<=
=> * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.<=
=> * See the License for the specific language governing permissions and<=
=> * limitations under the License.<=
=> */<=
=> import org.springframework.util.StringUtils; <=
=> 	 * Configure the name of a header that a subscription message can have for <

# Scope

In [247]:
modification_ranges = {}
for vuln in vulns_info:
    modification_ranges[vuln["id"]] = {}
    for file in affected_files[vuln["id"]]:
        show_vuln = vuln["repo"].git.show(vuln["fixing_commit"], "--", file).splitlines()
        added_ranges = []
        deleted_ranges = []
        for show in show_vuln:
            if "@@" in show:
                ranges = (show[3:])[:show[3:].find("@@")-1]
                deleted_range = ranges[1:ranges.find(" ")].split(",")
                deleted_range[0] = int(deleted_range[0])
                deleted_range[1] = deleted_range[0]+int(deleted_range[1])-1
                added_range = ranges[ranges.find("+")+1:].split(",")
                added_range[0] = int(added_range[0])
                added_range[1] = added_range[0]+int(added_range[1])-1
                added_ranges.append(added_range)
                deleted_ranges.append(deleted_range)
        modification_ranges[vuln["id"]][file] = {"added_ranges": added_ranges, "deleted_ranges": deleted_ranges}

In [248]:
modified_lines = {}
for vuln in vulns_info:
    modified_lines[vuln["id"]] = {}
    for file in modification_ranges[vuln["id"]]:
        show = vuln["repo"].git.show(vuln["fixing_commit"], "--", file).splitlines()
        added_filtered_lines = []
        deleted_filtered_lines = []
        for line in show:
            if line:
                if line[0] == "+":
                    if line[:3] == "+++":
                        continue
                    added_filtered_lines.append(line)
                if line[0] == "-":
                    if line[:3] == "---":
                        continue
                    deleted_filtered_lines.append(line)
        modified_lines[vuln["id"]][file] = {"added_lines": added_filtered_lines, "deleted_lines": deleted_filtered_lines}

## Getting Latest Commits for Deleted Lines

In [340]:
import copy
from collections import Counter

modified_lines_copy = copy.deepcopy(modified_lines)
deleted_commits = {}
for vuln in vulns_info:
    deleted_commits[vuln["id"]] = {}
    for file in modification_ranges[vuln["id"]]:
        for deleted_range in modification_ranges[vuln["id"]][file]["deleted_ranges"]:
            if deleted_range != [0, -1]:
                blame_lines = vuln["repo"].git.blame(vuln["fixing_commit"]+"^1", "-L", str(deleted_range[0])+","+str(deleted_range[1]), "--", file).splitlines()
            for line in blame_lines:
                line_content = line[line.find(") ")+2:].strip()
                found_line = None
                for deleted_line in modified_lines_copy[vuln["id"]][file]["deleted_lines"]:
                    if line_content in deleted_line[2:]:
                        found_line = deleted_line
                        break
                if found_line != None:
                    modified_lines_copy[vuln["id"]][file]["deleted_lines"].remove(found_line)
                    full_hash = vuln["repo"].git.rev_parse(line[:line.find(" ")])
                    if file in deleted_commits[vuln["id"]]:
                        deleted_commits[vuln["id"]][file].append(full_hash)
                    else:
                        deleted_commits[vuln["id"]][file] = [full_hash]

# Counting commits
counted_deleted_commits = {}
for v in deleted_commits:
    print('Latest Commits for Del:')
    counted_deleted_commits[v] = {}
    for file in deleted_commits[v]:
        counted_commits_for_file = Counter(deleted_commits[v][file])
        if file in counted_deleted_commits[v]:
            counted_deleted_commits[v][file].append(counted_commits_for_file)
        else:
            counted_deleted_commits[v][file] = [counted_commits_for_file]
        print(Grey + '------------------------------------------------------------')
        print(BLUE + "    --->", file)
        for commit, count in zip(counted_commits_for_file.keys(), counted_commits_for_file.values()):
            print(Grey + '------------------------------------------------------------')
            print(RED + "  count --->", count, "commit --->", commit)
            

Latest Commits for Del:
------------------------------------------------------------
    ---> spring-messaging/src/main/java/org/springframework/messaging/simp/broker/DefaultSubscriptionRegistry.java
------------------------------------------------------------
  count ---> 1 commit ---> 34a0857628d66c2b79214a37c52084597a743365
------------------------------------------------------------
  count ---> 4 commit ---> 3a2f5e71b7867328e325f773790ac2b2c10f78b9
------------------------------------------------------------
  count ---> 16 commit ---> b6327acec825aefadead62bd7825425b048b214c
------------------------------------------------------------
  count ---> 1 commit ---> b5c11ffb1cea98eb758534d9ca80101a58f4bbf8
------------------------------------------------------------
  count ---> 1 commit ---> 922e6de7886e2f18e8b8e2b6e60d34dbe69efe92
------------------------------------------------------------
  count ---> 1 commit ---> c879796e0092a188fa54117639067cfb29a2e858
-------------------------

## Getting Latest Commits for Added Lines

In [341]:
import copy
from collections import Counter

modified_lines_copy = copy.deepcopy(modified_lines)
added_commits = {}
for vuln in vulns_info:
    added_commits[vuln["id"]] = {}
    for file in modification_ranges[vuln["id"]]:
        full_file = vuln["repo"].git.blame(vuln["fixing_commit"], "--", file).splitlines()
        for added_range in modification_ranges[vuln["id"]][file]["added_ranges"]:
            if added_range != [0, -1]:
                blame_lines = vuln["repo"].git.blame(vuln["fixing_commit"], "-L", str(added_range[0])+","+str(added_range[1]), "--", file).splitlines()
            for line in blame_lines:
                line_content = line[line.find(") ")+2:].strip()
                found_line = None
                for added_line in modified_lines_copy[vuln["id"]][file]["added_lines"]:
                    if line_content in added_line[2:]:
                        found_line = added_line
                        break
                if found_line != None:
                    modified_lines_copy[vuln["id"]][file]["added_lines"].remove(found_line)
                    # Finding smallest enclosing scope
                    full_file_index = int(line[line.rfind(" ", 0,line.find(")")):line.find(")")]) - 1
                    ind = full_file_index-1
                    exited = False
                    noSkip = True
                    while ind >= 0:
                        full_file_line_content = full_file[ind][line.find(") ")+2:].strip()
                        if full_file_line_content.endswith("}"):
                            noSkip = False
                        if full_file_line_content.endswith("{") and not noSkip:
                            noSkip = True
                        elif full_file_line_content.endswith("{") and noSkip:
                            full_hash = vuln["repo"].git.rev_parse(full_file[ind][:full_file[ind].find(" ")])
                            if file in added_commits[vuln["id"]]:
                                added_commits[vuln["id"]][file].append(full_hash)
                            else:
                                added_commits[vuln["id"]][file] = [full_hash]
                            exited = True
                            break
                        ind -= 1
                    if ind < 0 and not exited:
                        full_hash = vuln["repo"].git.rev_parse(line[:line.find(" ")])
                        if file in added_commits[vuln["id"]]:
                            added_commits[vuln["id"]][file].append(full_hash)
                        else:
                            added_commits[vuln["id"]][file] = [full_hash]

# Counting commits
counted_added_commits = {}
for v in added_commits:
    print('Latest commits for add:')
    counted_added_commits[v] = {}
    for file in added_commits[v]:
        counted_commits_for_file = Counter(added_commits[v][file])
        if file in counted_added_commits[v]:
            counted_added_commits[v][file].append(counted_commits_for_file)
        else:
            counted_added_commits[v][file] = [counted_commits_for_file]
        print(Grey + '------------------------------------------------------------')
        print(BLUE + "    --->", file)
        for commit, count in zip(counted_commits_for_file.keys(), counted_commits_for_file.values()):
            print(Grey + '------------------------------------------------------------')
            print(RED + "  count --->", count, "commit --->", commit)

Latest commits for add:
------------------------------------------------------------
    ---> spring-messaging/src/main/java/org/springframework/messaging/simp/broker/DefaultSubscriptionRegistry.java
------------------------------------------------------------
  count ---> 14 commit ---> 246a6db1cad205ca9b6fca00c544ab7443ba202c
------------------------------------------------------------
  count ---> 11 commit ---> 3a2f5e71b7867328e325f773790ac2b2c10f78b9
------------------------------------------------------------
  count ---> 1 commit ---> b5c11ffb1cea98eb758534d9ca80101a58f4bbf8
------------------------------------------------------------
  count ---> 2 commit ---> b6327acec825aefadead62bd7825425b048b214c
------------------------------------------------------------
  count ---> 4 commit ---> 34a0857628d66c2b79214a37c52084597a743365
------------------------------------------------------------
    ---> spring-messaging/src/main/java/org/springframework/messaging/simp/broker/SimpleBrok

## Getting VCCs

In [349]:
from collections import Counter
count_the_most = 0
vcc_commit = ""
i = 0
VCC = {}

# Counting commits
for v in deleted_commits:
    VCC[v] = {}
    print("Result Cal:" + '\033[95m')
    print('--------------------------------------------------------------------------------------------------------')
    for f in added_commits[v]:
        print("    --------->", f)
        print('--------------------------------------------------------------------------------------------------------')
        if f in deleted_commits[v] and f in added_commits[v]:
            counted_commits = Counter(deleted_commits[v][f]+added_commits[v][f])
        elif f in deleted_commits[v] and f not in added_commits[v]:
            counted_commits = Counter(deleted_commits[v][f])
        elif f not in deleted_commits[v] and f in added_commits[v]:
            counted_commits = Counter(added_commits[v][f])
        VCC[v][f] = {"count": 0, "commit": ""}
        for commit, count in zip(counted_commits.keys(), counted_commits.values()):
            if (count > VCC[v][f]["count"]):
                VCC[v][f] = {"count": count, "commit": commit}
            elif (count == VCC[v][f]["count"]):
                VCC[v][f]["count"] = count
        print("  count  ---> ", VCC[v][f]["count"], "commit ---> ", VCC[v][f]["commit"])
        print('--------------------------------------------------------------------------------------------------------')
        
        if (count_the_most <= VCC[v][f]["count"]):
            count_the_most = VCC[v][f]["count"]
            vcc_commit = VCC[v][f]["commit"];
           

print('\n')
print(Cyan + "Finial Result:" + Cyan)
print('---------> ' + vcc_commit)

        

Result Cal:
--------------------------------------------------------------------------------------------------------
    ---------> spring-messaging/src/main/java/org/springframework/messaging/simp/broker/DefaultSubscriptionRegistry.java
--------------------------------------------------------------------------------------------------------
  count  --->  18 commit --->  b6327acec825aefadead62bd7825425b048b214c
--------------------------------------------------------------------------------------------------------
    ---------> spring-messaging/src/main/java/org/springframework/messaging/simp/broker/SimpleBrokerMessageHandler.java
--------------------------------------------------------------------------------------------------------
  count  --->  26 commit --->  4c0da5867a54d3f81098970538b9795af732d837
--------------------------------------------------------------------------------------------------------
    ---------> spring-messaging/src/main/java/org/springframework/messaging/si

# Prep

In [252]:
import numpy as np
import pandas as pd
import os

# Specify git executable file for GitPython in Jupyter Notebook (In IDE, it can still work without this line.)
os.environ["GIT_PYTHON_GIT_EXECUTABLE"] = "C:\Program Files\Git\cmd\git.exe"

import git
from git import RemoteProgress

from git import Repo
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

In [271]:
class Progress(RemoteProgress):
    def update(self, op_code, cur_count, max_count=None, message=''):
        print(self._cur_line)

In [272]:
local_link = "spring-framework"
repo = Repo(local_link)
fixing_commit = "4c0da5867a54d3f81098970538b9795af732d837"
previous = fixing_commit + '^'

In [273]:
# Question a: show commit title and commit msg

In [274]:
show_data = repo.git.log("--format=%B", "-n", "1",fixing_commit).splitlines()

for line in show_data:
    print(line)

Add Java config support for WebSocket and STOMP

Issue: SPR-10835


In [275]:
# Question b: Show all affected files

In [276]:
 affected_file = repo.git.diff( "--name-only", fixing_commit, previous).splitlines()

print(len(affected_file))

31


In [277]:
# Question c: show all directories are affected

In [351]:
affected_directories = repo.git.diff("--dirstat", fixing_commit, previous).splitlines()
print("The number of dirs are affected:")
print(len(affected_directories))

The number of dirs are affected:
7


In [279]:
# Question d: How many lines were deleted?

In [280]:
sum_delete = 0;
# print(previous)
deleted_lines = repo.git.diff(previous, fixing_commit).splitlines()
rule = re.compile('^-$|^-[^-]')

for line in deleted_lines:
    if rule.match(line):
        sum_delete = sum_delete + 1

            
print("Lines deleted: %d" %sum_delete)

#     print(line)

Lines deleted: 290


In [281]:
# Question e: How many lines were added?

In [282]:
sum_add = 0;
# print(previous)
add_lines = repo.git.diff(previous, fixing_commit).splitlines()
rule = re.compile('^\+$|^\+[^\+]')

for line in deleted_lines:
        if rule.match(line):
            sum_add += 1
       
    
print("This is the added lines including comments and blank lines")
print("Lines added: %d" %sum_add)


This is the added lines including comments and blank lines
Lines added: 2236


In [283]:
# Question f: How many lines were deleted (Excluding blank line & comment)

In [284]:
sum_deleted_without = 0;
# print(previous)
delete_lines = repo.git.diff("--ignore-blank-lines",previous, fixing_commit).splitlines()
rule = re.compile('^-$|^-[^-]')
rule1 = re.compile('^[-|\+]\s*\*')
rule2 = re.compile('^[-|\+]\s*\/\/')

for line in deleted_lines:
        if rule.match(line):
            sum_deleted_without += 1
            if rule1.match(line) or rule2.match(line):
                sum_deleted_without -= 1
            
print("Lines added: %d" %sum_deleted_without)

Lines added: 282


In [285]:
# Question g: How many lines were added (Excluding blank line & comment)

In [286]:
sum_added_without = 0;
# print(previous)
add_lines = repo.git.diff("--ignore-blank-lines",previous, fixing_commit).splitlines()
rule = re.compile('^\+$|^\+[^\+]')
rule1 = re.compile('^[-|\+]\s*\*')
rule2 = re.compile('^[-|\+]\s*\/\/')

for line in add_lines:
        if rule.match(line):
            sum_added_without += 1
            if rule1.match(line) or rule2.match(line):
                sum_added_without -= 1
            
print("Lines added: %d" %sum_added_without)

Lines added: 1657


In [287]:
# Question h: How many days were between the current fixing commit and the previous commit of each affected file

In [288]:
time_different = 0
sum = 0
count = 0
for line in repo.git.diff("--name-only", fixing_commit, previous).splitlines():
    creation = repo.git.log("-2", "--pretty=%ct","--", line).splitlines();
    if len(creation) == 2:
        count += 1;
        time_different = ((float(creation[0]) - float(creation[1])) / 86400) # 86400 is one day
        sum = sum + time_different
        print("%s \t%.1f"%(line,((float(creation[0]) - float(creation[1])) / 86400)))
    else:
        print("%s is New file" %line)

avg = sum / count
print("%.1f" %avg)

build.gradle 	0.0
spring-messaging/src/main/java/org/springframework/messaging/handler/websocket/SubProtocolWebSocketHandler.java 	87.0
spring-messaging/src/main/java/org/springframework/messaging/simp/config/AbstractBrokerRegistration.java 	652.7
spring-messaging/src/main/java/org/springframework/messaging/simp/config/DelegatingWebSocketMessageBrokerConfiguration.java 	88.0
spring-messaging/src/main/java/org/springframework/messaging/simp/config/EnableWebSocketMessageBroker.java 	13.4
spring-messaging/src/main/java/org/springframework/messaging/simp/config/MessageBrokerConfigurer.java 	13.4
spring-messaging/src/main/java/org/springframework/messaging/simp/config/SimpleBrokerRegistration.java 	339.0
spring-messaging/src/main/java/org/springframework/messaging/simp/config/StompBrokerRelayRegistration.java 	154.3
spring-messaging/src/main/java/org/springframework/messaging/simp/config/StompEndpointRegistration.java 	82.5
spring-messaging/src/main/java/org/springframework/messaging/simp/c

In [289]:
# Question i: How many times has each affected file of the current fixing commit been modified in the past since their creation (including rename of the file)?

In [290]:
sum_time = 0
count = 0
# implement code
for line in repo.git.diff("--name-only", fixing_commit, previous).splitlines():
    count = count + 1;
    times = repo.git.log("--follow", "--pretty=oneline", "--", line).splitlines();
    sum_time = sum_time + len(times)
    print(" %s: %d " %(line, len(times)))

avg = sum_time / count
print("%d" %avg)

 build.gradle: 1302 
 spring-messaging/src/main/java/org/springframework/messaging/handler/websocket/SubProtocolWebSocketHandler.java: 5 
 spring-messaging/src/main/java/org/springframework/messaging/simp/config/AbstractBrokerRegistration.java: 13 
 spring-messaging/src/main/java/org/springframework/messaging/simp/config/DelegatingWebSocketMessageBrokerConfiguration.java: 2 
 spring-messaging/src/main/java/org/springframework/messaging/simp/config/EnableWebSocketMessageBroker.java: 3 
 spring-messaging/src/main/java/org/springframework/messaging/simp/config/MessageBrokerConfigurer.java: 7 
 spring-messaging/src/main/java/org/springframework/messaging/simp/config/SimpleBrokerRegistration.java: 14 
 spring-messaging/src/main/java/org/springframework/messaging/simp/config/StompBrokerRelayRegistration.java: 18 
 spring-messaging/src/main/java/org/springframework/messaging/simp/config/StompEndpointRegistration.java: 4 
 spring-messaging/src/main/java/org/springframework/messaging/simp/confi

In [291]:
# Question j: Which developers have modified each affected file since its creation? 

In [292]:
count = 0
for line in repo.git.diff("--name-only", fixing_commit, previous).splitlines():
    print(" %s"%line)
    authors = repo.git.log("--follow", "--pretty=%aN", "--", line).splitlines()
    for author in set(authors): # set for not repeating the same author
        print(" %s" %author)
        count = count + 1;

avg = count / len(affected_file)
print("%.1f" %avg)

 build.gradle
 Raman Gupta
 Marius Grama
 Stephane Nicoll
 Christian Dupuis
 Stephane Maldini
 Rossen Stoyanchev
 Kris De Volder
 Jeremy Grelle
 Andy Wilkinson
 Vedran Pavic
 Violeta Georgieva
 Arjen Poutsma
 Roy Clarkson
 Fredrik Sundberg
 sdeleuze
 Craig Andrews
 Glyn Normington
 Eddú Meléndez
 Rob Winch
 Phillip Webb
 Lars Grefer
 Brian Clozel
 Chris Beams
 Spring Operator
 Thor Andreas Rognan
 D瓜哥
 Oliver Gierke
 Ben Manes
 Sam Brannen
 Stevo Slavic
 Mario Arias
 Sebastien Deleuze
 Luciano Leggieri
 Dave Syer
 Juergen Hoeller
 spring-messaging/src/main/java/org/springframework/messaging/handler/websocket/SubProtocolWebSocketHandler.java
 Andy Wilkinson
 Rossen Stoyanchev
 spring-messaging/src/main/java/org/springframework/messaging/simp/config/AbstractBrokerRegistration.java
 Brian Clozel
 Sebastien Deleuze
 Rossen Stoyanchev
 Spring Operator
 Phillip Webb
 Juergen Hoeller
 spring-messaging/src/main/java/org/springframework/messaging/simp/config/DelegatingWebSocketMessageBrokerConf

In [293]:
# Question k:

In [294]:
allAuthors = []
count = 0
for line in repo.git.diff("--name-only", fixing_commit, previous).splitlines():
    allAuthors += repo.git.log("--follow", "--pretty=%aN", "--", line).splitlines()
allAuthors = list(set(allAuthors))
log = repo.git.log("--pretty=%aN")
for author in allAuthors:
    count = count + 1
    print(" %s: %d" %(author, len(re.findall(author, log))))
    
print("%d" %count)

 Raman Gupta: 1
 Marius Grama: 1
 Stephane Nicoll: 776
 Christian Dupuis: 24
 Stephane Maldini: 115
 Rossen Stoyanchev: 3323
 Kris De Volder: 1
 Jeremy Grelle: 15
 Andy Wilkinson: 53
 Vedran Pavic: 8
 Violeta Georgieva: 54
 Arjen Poutsma: 1212
 Roy Clarkson: 4
 Fredrik Sundberg: 3
 sdeleuze: 52
 Craig Andrews: 5
 Glyn Normington: 2
 diguage: 10
 Eddú Meléndez: 5
 Rob Winch: 86
 Phillip Webb: 611
 Lars Grefer: 4
 Brian Clozel: 564
 Chris Beams: 1049
 Spring Operator: 8
 Thor Andreas Rognan: 2
 D瓜哥: 1
 Oliver Gierke: 36
 Ben Manes: 1
 Sam Brannen: 2505
 Stevo Slavic: 16
 Mario Arias: 2
 Sebastien Deleuze: 742
 Luciano Leggieri: 1
 Eric Dahl: 1
 Dave Syer: 9
 Mark Pollack: 75
 Jan Machacek: 2
 Juergen Hoeller: 5998
 Johnny Lim: 44
40
